In [ ]:
from ecmwfapi import *
import xarray as xr
import cdsapi
import os
import zipfile
import shutil

In [1]:
# Download past IFS-HRES forecast from mars: all surface parameters (sv)
# Parameters: 2m temperature, 10m U wind component, 10m V wind component, Mean sea level pressure
# Months 1 - 12 of 2024, First 240 hours (10 days) of each month

In [ ]:
# Initialize the ECMWFService
server = ECMWFService("mars")

# List of months for the year 2024
months = [f"{m:02d}" for m in range(1, 13)]  # ["01", "02", ..., "12"]
year = "2024"
day = "01"

for month in months:
    date_str = f"{year}{month}{day}"
    target_file = f"Surface Variables/{date_str}/{date_str}_marsfc_sv.nc"
    
    request_params = {
        "class": "od",
        "date": date_str,
        "expver": "1",
        "levtype": "sfc",
        "param": "165.128/166.128/167.128/151.128",
        "step": "0/to/240/by/6",
        "stream": "oper",
        "time": "00",
        "format": "netcdf",
        "type": "fc",
        "grid": "0.25/0.25"
    }
    
    print(f"Requesting data for {date_str}...")
    server.execute(request_params, target_file)
    print(f"Data saved to {target_file}\n")


In [ ]:
# Download past AIFS forecast from mars: all surface parameters (sv)
# Parameters: 2m temperature, 10m U wind component, 10m V wind component, Mean sea level pressure
# Months 1 - 12 of 2024, First 240 hours (10 days) of each month

In [ ]:
# Initialize the ECMWFService
server = ECMWFService("mars")

# List of months for the year 2024
months = [f"{m:02d}" for m in range(1, 13)]  # ["01", "02", ..., "12"]
year = "2024"
day = "01"

for month in months:
    date_str = f"{year}{month}{day}"
    target_file = f"Surface Variables/{date_str}/{date_str}_marsai_sv.nc"
    
    request_params = {
        "class": "ai",
        "date": date_str,
        "expver": "1",
        "levtype": "sfc",
        "model" : "aifs-single",
        "param": "151/165/166/167",
        "step": "0/to/240/by/6",
        "stream": "oper",
        "time": "06",
        "format": "netcdf",
        "type": "fc",
        "grid": "0.25/0.25"
    }
    
    print(f"Requesting data for {date_str}...")
    server.execute(request_params, target_file)
    print(f"Data saved to {target_file}\n")

In [ ]:
# Download ERA5 reanalysis from CDS: all surface parameters (sv)
# Parameters: 2m temperature, 10m U wind component, 10m V wind component, Mean sea level pressure
# Months 1 - 12 of 2024, First 240 hours (10 days) of each month

In [ ]:
# Initialize the CDS API client
client = cdsapi.Client(url="https://cds.climate.copernicus.eu/api", key="")

# Define the dataset
dataset = "reanalysis-era5-single-levels"

# Define the months for the year 2024
months = [f"{m:02d}" for m in range(1, 13)]  # ["01", "02", ..., "12"]

# Loop through each month and make a request
for month in months:
    request = {
        "product_type": ["reanalysis"],
        "variable": [
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "2m_temperature",
            "mean_sea_level_pressure",

        ],
        "year": ["2024"],
        "month": [month],  # Dynamically set the month
        "day": [
            "01", "02", "03",
            "04", "05", "06",
            "07", "08", "09",
            "10", "11"
            ],  
        "time": ["00:00", "06:00", "12:00", "18:00"],
        "data_format": "netcdf",
        "download_format": "unarchived" # eventhough "unarchived" is declared, script downloads a zip file
    }

    # Define output file name based on the date
    output_filename = f"Surface Variables/2024{month}01_era5_sv.zip"

    # Make the request and download the data
    print(f"Requesting data for 2024-{month}-01...")
    client.retrieve(dataset, request).download(output_filename)
    print(f"Download complete: {output_filename}")

print("All downloads completed!")

In [ ]:
# Turn the ERA 5 zip files into NetCDF

In [ ]:
# Define the months for the year 2024
months = [f"{m:02d}" for m in range(1, 13)]  # ["01", "02", ..., "12"]

# Loop through each month's ZIP file
for month in months:
    # Define file paths dynamically
    zip_path = f"Surface Variables/2024{month}01_era5_sv.zip"
    extract_path = "Surface Variables/extracted_nc"
    output_file = f"Surface Variables/2024{month}01_era5_sv.nc"

    # Ensure extraction directory exists
    os.makedirs(extract_path, exist_ok=True)

    # Extract files from ZIP
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    
    # Get a list of extracted NetCDF files
    nc_files = [os.path.join(extract_path, f) for f in zip_ref.namelist() if f.endswith(".nc")]
    print(f"Extracted NetCDF files for 2024-{month}-01: {nc_files}")

    # Open and print variable names from each NetCDF file
    datasets = []
    for nc_file in nc_files:
        ds = xr.open_dataset(nc_file)
        print(f"Variables in {nc_file}: {list(ds.data_vars.keys())}")
        datasets.append(ds)

    # Merge both NetCDF datasets
    ds_combined = xr.merge(datasets)

    # Save the merged dataset
    ds_combined.to_netcdf(output_file)
    print(f"Merged dataset saved as: {output_file}")

    # ===== CLEANUP: DELETE EXTRACTED FILES AND ZIP FILE =====
    shutil.rmtree(extract_path)  # Delete extracted_nc folder
    os.remove(zip_path)  # Delete the original ZIP file
    print(f"Cleanup complete for 2024-{month}-01.")

print("All decompression tasks completed!")